In [ ]:
%%capture
!pip install -U git+https://github.com/UN-GCPDS/python-gcpds.databases #Package for database reading.
!pip install mne #The MNE Package is installed
FILEID = "1lo0MjWLvsyne2CgTA6VZ2HGY9SKxiwZ7"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MI_EEG_ClassMeth.zip && rm -rf /tmp/cookies.txt
!unzip MI_EEG_ClassMeth.zip #Package with useful functions for motor imagery classification based in EEG.
!dir

In [ ]:
!apt-get install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 -y
!pip install tensorflow==2.8.2

In [ ]:
from gcpds.databases import GIGA_MI_ME
from typing import Sequence, Tuple
from MI_EEG_ClassMeth.FeatExtraction import TimeFrequencyRpr
import numpy as np
from scipy.signal import resample

def load_GIGA(db: GIGA_MI_ME,
              sbj: int,
              eeg_ch_names: Sequence[str],
              fs: float, 
              f_bank: np.ndarray, 
              vwt: np.ndarray, 
              new_fs: float) -> Tuple[np.ndarray, np.ndarray]:

  index_eeg_chs = db.format_channels_selectors(channels = eeg_ch_names) - 1

  tf_repr = TimeFrequencyRpr(sfreq = fs, f_bank = f_bank, vwt = vwt)

  db.load_subject(sbj)
  X, y = db.get_data(classes = ['left hand mi', 'right hand mi']) #Load MI classes, all channels {EEG}, reject bad trials, uV
  X = X[:, index_eeg_chs, :] #spatial rearrangement
  X = np.squeeze(tf_repr.transform(X))
  #Resampling
  if new_fs == fs:
    print('No resampling, since new sampling rate same.')
  else:
    print("Resampling from {:f} to {:f} Hz.".format(fs, new_fs))
    X = resample(X, int((X.shape[-1]/fs)*new_fs), axis = -1)
    
  return X, y

from tensorflow.keras.layers import Layer
import tensorflow as tf
import tensorflow_probability as tfp

class GFC(Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def build(self, batch_input_shape):
    self.gammad = self.add_weight(name = 'gammad',
                            shape = (),
                            initializer = 'zeros',
                            trainable = True)
    super().build(batch_input_shape)

  def call(self, X): 
    X = tf.transpose(X, perm  = (0, 3, 1, 2)) #(N, F, C, T)
    R = tf.reduce_sum(tf.math.multiply(X, X), axis = -1, keepdims = True) #(N, F, C, 1)
    D  = R - 2*tf.matmul(X, X, transpose_b = (0, 1, 3, 2)) + tf.transpose(R, perm = (0, 1, 3, 2)) #(N, F, C, C)

    ones = tf.ones_like(D[0,0,...]) #(C, C)
    mask_a = tf.linalg.band_part(ones, 0, -1) #Upper triangular matrix of 0s and 1s (C, C)
    mask_b = tf.linalg.band_part(ones, 0, 0)  #Diagonal matrix of 0s and 1s (C, C)
    mask = tf.cast(mask_a - mask_b, dtype=tf.bool) #Make a bool mask (C, C)
    triu = tf.expand_dims(tf.boolean_mask(D, mask, axis = 2), axis = -1) #(N, F, C*(C-1)/2, 1)
    sigma = tfp.stats.percentile(tf.math.sqrt(triu), 50, axis = 2, keepdims = True) #(N, F, 1, 1)

    A = tf.math.exp(-1/(2*tf.pow(10., self.gammad)*tf.math.square(sigma))*D) #(N, F, C, C)
    A.set_shape(D.shape)
    return A

  def compute_output_shape(self, batch_input_shape):
    N, C, T, F = batch_input_shape.as_list()
    return tf.TensorShape([N, F, C, C])

  def get_config(self):
    base_config = super().get_config()
    return {**base_config}

from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, DepthwiseConv2D, Activation, AveragePooling2D, Dropout, SpatialDropout2D, SeparableConv2D, Flatten, Dense
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Model

def get_model(number_of_classes: int,
          number_of_channels: int,
          number_of_time_samples: int, 
          dropout_rate: float,
          kernLength: int,
          F1: int, 
          D: int,
          F2: int,
          norm_rate: int,
          dropout_type: str) -> Model:
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
	advised to do some model searching to get optimal performance on your
	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      number_of_channels, number_of_time_samples  : number of channels and time points in the EEG data
      dropout_rate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropout_type     : Either SpatialDropout2D or Dropout, passed as a string.
    """
    
    if dropout_type == 'SpatialDropout2D':
        dropout_type = SpatialDropout2D
    elif dropout_type == 'Dropout':
        dropout_type = Dropout
    else:
        raise ValueError('dropout_type must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input_   = Input(shape = (number_of_channels, number_of_time_samples, 1))

    ##################################################################
    #Temporal Convolution
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                          name = 'Temporal_Conv2D',
                          use_bias = False)(input_)
    block1       = BatchNormalization()(block1)
    adj_mat      = GFC(name = 'gfc')(block1)
    
    ##################################################################
    #Spatial Convolution
    block1       = DepthwiseConv2D((number_of_channels, 1),
                                   name = 'Spatial_Depth_wise_Conv2D',
                                   depth_multiplier = D,
                                   use_bias = False, 
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropout_type(dropout_rate)(block1)
    
    ##################################################################
    #Separable Convolution
    block2       = SeparableConv2D(F2, (1, 16), padding = 'same',
                                   name = 'Separable_Conv2D',
                                   use_bias = False)(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropout_type(dropout_rate)(block2)
    
    ##################################################################
    # Classification block
    flatten      = Flatten(name = 'flatten')(block2)
    dense        = Dense(number_of_classes, name = 'output', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'out_activation')(dense)
    
    return Model(inputs=input_, outputs = [softmax, adj_mat])

from typing import Callable
import tensorflow as tf
import keras.backend as K

def adjacency_matrix_regularization(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
    global N
    F = y_pred.shape[1]
    C = y_pred.shape[-1]
    scalar_kernely = tfp.math.psd_kernels.ExponentiatedQuadratic(amplitude = 1, length_scale = 1e-13)
    y_true = tf.cast(y_true, dtype = tf.float32)
    y_true = tf.expand_dims(y_true, axis = -1) #(N, 1)
    Ky = scalar_kernely.matrix(y_true, y_true) #(N, N)
    #N = Ky.shape[-1]
    Ky = tf.expand_dims(Ky, axis = 0) #(1, N, N)
    ones = tf.ones_like(y_pred[0, 0, ...])
    mask_a = tf.linalg.band_part(ones, 0, -1) #Upper triangular matrix of 0s and 1s
    mask_b = tf.linalg.band_part(ones, 0, 0)  #Diagonal matrix of 0s and 1s
    mask = tf.cast(mask_a - mask_b, dtype = tf.bool) #Make a bool mask
    A = tf.boolean_mask(y_pred, mask, axis = 2) #(N, F, C(C - 1)/2)
    A.set_shape([N, F, int(C*(C - 1)*0.5)])
    A = tf.linalg.normalize(A, ord = 'euclidean', axis = -1)[0] #Normalize connectivities vectors
    A = tf.transpose(A, perm  = (1, 0, 2)) #(F, N, C(C - 1)/2)
    Ka = tf.linalg.matmul(A, A, transpose_b = True) #(F, N, N)
    h = tf.eye(N) - (1.0/N)*tf.ones([N,1])*tf.ones([1,N]) #matrix for centered kernel
    trkl = tf.linalg.trace(tf.matmul(tf.matmul(Ka,h), tf.matmul(Ky,h)))
    trkk = tf.linalg.trace(tf.matmul(tf.matmul(Ka,h),tf.matmul(Ka,h)))
    trll = tf.linalg.trace(tf.matmul(tf.matmul(Ky,h),tf.matmul(Ky,h)))
    loss = tf.math.reduce_sum(-trkl/tf.sqrt(trkk*trll))/F
    return loss

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN
import numpy as np
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit
from os import makedirs

db = GIGA_MI_ME('/kaggle/input/giga-science-gcpds/GIGA_MI_ME')
fs = db.metadata['sampling_rate']
eeg_ch_names = ['Fp1','Fpz','Fp2',
              'AF7','AF3','AFz','AF4','AF8',
              'F7','F5','F3','F1','Fz','F2','F4','F6','F8',
              'FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8',
              'T7','C5','C3','C1','Cz','C2','C4','C6','T8',
              'TP7','CP5','CP3','CP1','CPz','CP2','CP4','CP6','TP8',
              'P9','P7','P5','P3','P1','Pz','P2','P4','P6','P8','P10',
              'PO7','PO3','POz','PO4','PO8',
              'O1','Oz','O2',
              'Iz']
verbose = 0
reduce_lr_on_plateau = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, patience = 30, verbose = verbose, mode = 'min', min_delta = 0.01, min_lr = 0)
terminate_on_nan = TerminateOnNaN()
callbacks = [reduce_lr_on_plateau, terminate_on_nan]
seed = 23

load_args = dict(db = db,
                 eeg_ch_names = eeg_ch_names,
                 fs = fs,
                 f_bank = np.asarray([[4., 40.]]),
                 vwt = np.asarray([[2.5, 5]]),
                 new_fs = 128.)

cv_args = dict(hyparams = [0, 0.2, 0.4, 0.6, 0.8],
               cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2, random_state = seed))

model_args = dict(number_of_classes = 2,
                  dropout_rate = 0.5,
                  kernLength = int(load_args['new_fs']/4),
                  F1 = 8,
                  D = 2,
                  F2 = 16,
                  norm_rate = 0.25,
                  dropout_type = 'Dropout')

compile_args = dict(loss = [SparseCategoricalCrossentropy(), adjacency_matrix_regularization],
                    init_lr = 1e-2)
                        
fit_args = dict(epochs = 500,
                verbose = verbose,
                callbacks = callbacks)

In [ ]:
from tensorflow.random import set_seed
from tensorflow.keras.backend import clear_session
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score

def train(load_args, cv_args, model_args, compile_args, fit_args, seed):
    global N
    X_train, y_train = load_GIGA(**load_args)
    X_train = X_train[..., np.newaxis]

    cv_results = {'params': [], 'mean_acc':[], 'std_acc':[],
                  'mean_kappa':[], 'std_kappa':[],
                  'mean_auc':[], 'std_auc':[]}
    # add measurements

    for lambda_ in cv_args['hyparams']:
      cv_results['params'].append(lambda_)
      acc = np.zeros(cv_args['cv'].get_n_splits())
      kappa = np.zeros(cv_args['cv'].get_n_splits())
      auc = np.zeros(cv_args['cv'].get_n_splits())
        # init vectors
      k = 0
      for train_index, val_index in cv_args['cv'].split(X_train, y_train):
        X, X_val = X_train[train_index], X_train[val_index]
        y, y_val = y_train[train_index], y_train[val_index]

        batch_size, C, T = X.shape[:-1]
        
        N = batch_size

        clear_session()
        set_seed(seed)

        model = get_model(**model_args, number_of_channels = C, number_of_time_samples = T)
        model.compile(loss = compile_args['loss'], 
                      loss_weights = [1 - lambda_, lambda_],
                      optimizer = Adam(compile_args['init_lr']))
        
        history = model.fit(X, [y, y],
                  batch_size = batch_size,
                  **fit_args)

        y_prob = model.predict(X_val)[0]
        y_pred = np.argmax(y_prob, axis = 1)

        acc[k] = accuracy_score(y_val, y_pred)
        kappa[k] = cohen_kappa_score(y_val, y_pred)
        auc[k] = roc_auc_score(y_val, y_prob[:, 1], average = 'macro')
        # f1, recall, preci (0, 1)

        print('lambda = ', lambda_, ', Fold =  ', k+1, ', acc = ', acc[k])
        k += 1

      cv_results['mean_acc'].append(round(acc.mean(), 3))
      cv_results['std_acc'].append(round(acc.std(), 3))
      cv_results['mean_kappa'].append(round(kappa.mean(), 3))
      cv_results['std_kappa'].append(round(kappa.std(), 3))
      cv_results['mean_auc'].append(round(auc.mean(), 3))
      cv_results['std_auc'].append(round(auc.std(), 3))

    best_score = max(cv_results['mean_acc'])
    index = [idx for idx, val in enumerate(cv_results['mean_acc']) if val == best_score]
    if len(index) == 1:
      best_index = index[0]
    else:
      min_std =  cv_results['std_acc'][index[0]]
      best_index = index[0]
      for idx in index[1:]:
        if min_std > cv_results['std_acc'][idx]:
          min_std = cv_results['std_acc'][idx]
          best_index = idx
        
    cv_results['best_index'] = best_index
            
    print('best param = ', cv_results['params'][cv_results['best_index']], ', best_score = ', best_score)

    #refit

    batch_size, C, T = X_train.shape[:-1]

    clear_session()
    set_seed(seed)

    model = get_model(**model_args, number_of_channels = C, number_of_time_samples = T)
    model.compile(loss = compile_args['loss'],
                  loss_weights = [1 - cv_results['params'][cv_results['best_index']], cv_results['params'][cv_results['best_index']]],
                  optimizer = Adam(compile_args['init_lr']))
    
    history = model.fit(X, [y, y],
              batch_size = batch_size,
              **fit_args)

    model.save_weights('sbj' + str(load_args['sbj']) +'.h5')

    return cv_results

In [ ]:
from pickle import dump

subjects = np.arange(db.metadata['subjects']) + 1
subjects = np.delete(subjects, [28,33])

for sbj in subjects[40:]:
    load_args['sbj'] = sbj
    results = train(load_args, cv_args, model_args, compile_args, fit_args, seed)
    with open('sbj' + str(load_args['sbj']) + '.txt', 'wb') as f:
        dump(results, f)

In [ ]:
!zip Models.zip ./*.h5
!zip Results.zip ./*.txt